# Prompt templates
source: https://python.langchain.com/docs/how_to/#prompt-templates

Langchain **prompt templates**  are responsible for formatting user input into a format that can be passed to a language model.


Prompt template types: 
- few-shot examples
- few-shot examples in chat models
- partially format prompt templates
- compose prompts together
- multimodal prompts


## Together prompts composition

**Basic idea**: format reusable prompts which contain variables passed as arguments when calling the prompt template.

In [2]:
# String prompt composition
from langchain_core.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny and in {language}"
)

prompt.format(topic="sports", language="spanish")

'Tell me a joke about sports, make it funny and in spanish'

In [3]:
# Chat prompt composition
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

prompt = SystemMessage(content="You are a nice pirate")

new_prompt = ( prompt + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}")
new_prompt.format_messages(input="i said hi")

[SystemMessage(content='You are a nice pirate', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
 AIMessage(content='what?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i said hi', additional_kwargs={}, response_metadata={})]

In [ ]:
# Creating a prompt template
from langchain.prompts import ChatPromptTemplate

messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    ("human", "Tell me {joke_count} jokes."),
]
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"topic": "lawyers", "joke_count": 3})

print("\n----- Prompt with System and Human Messages (Tuple) -----\n")
print(prompt)


----- Prompt with System and Human Messages (Tuple) -----

messages=[SystemMessage(content='You are a comedian who tells jokes about lawyers.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me 3 jokes.', additional_kwargs={}, response_metadata={})]


## Few shot prompt 
Basic idea: By providing the model with examples like this, we can guide the model to a better response.

In [4]:
# Create a formatter for the few-shot examples (this formatter will format the few-shot examples into a string)
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")

# Create example set
examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """Are follow up questions needed here: Yes.
                    Follow up: How old was Muhammad Ali when he died?
                    Intermediate answer: Muhammad Ali was 74 years old when he died.
                    Follow up: How old was Alan Turing when he died?
                    Intermediate answer: Alan Turing was 41 years old when he died.
                    So the final answer is: Muhammad Ali""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """Are follow up questions needed here: Yes.
                    Follow up: Who was the founder of craigslist?
                    Intermediate answer: Craigslist was founded by Craig Newmark.
                    Follow up: When was Craig Newmark born?
                    Intermediate answer: Craig Newmark was born on December 6, 1952.
                    So the final answer is: December 6, 1952""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """Are follow up questions needed here: Yes.
                    Follow up: Who was the mother of George Washington?
                    Intermediate answer: The mother of George Washington was Mary Ball Washington.
                    Follow up: Who was the father of Mary Ball Washington?
                    Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
                    So the final answer is: Joseph Ball""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """Are follow up questions needed here: Yes.
                    Follow up: Who is the director of Jaws?
                    Intermediate Answer: The director of Jaws is Steven Spielberg.
                    Follow up: Where is Steven Spielberg from?
                    Intermediate Answer: The United States.
                    Follow up: Who is the director of Casino Royale?
                    Intermediate Answer: The director of Casino Royale is Martin Campbell.
                    Follow up: Where is Martin Campbell from?
                    Intermediate Answer: New Zealand.
                    So the final answer is: No""",
    },
]

In [5]:
# Using the few-shot examples and the formatter, the FewShotPromptTemplate object formats the passed examples  
# and adds them to the final prompt before suffix:

from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input": "Who was the father of Mary Ball Washington?"}).to_string()
)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
                    Follow up: How old was Muhammad Ali when he died?
                    Intermediate answer: Muhammad Ali was 74 years old when he died.
                    Follow up: How old was Alan Turing when he died?
                    Intermediate answer: Alan Turing was 41 years old when he died.
                    So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
                    Follow up: Who was the founder of craigslist?
                    Intermediate answer: Craigslist was founded by Craig Newmark.
                    Follow up: When was Craig Newmark born?
                    Intermediate answer: Craig Newmark was born on December 6, 1952.
                    So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up question

In [7]:
# We now feed the prompt examples into an implementation of ExampleSelector (called SemanticSimilarityExampleSelector instance)
# This class selects few-shot examples from the initial set based on their similarity to the input. 
# It uses an embedding model to compute the similarity between the input and the few-shot examples, as well as a vector store to perform the nearest neighbor search.
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
# from langchain.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,   
    # embedding class used to produce embeddings which are used to measure semantic similarity.
    OllamaEmbeddings(base_url="http://localhost:11434", model="llama3.2:3b"),
    # VectorStore class used to store the embeddings and do a similarity search over.
    Chroma,  
    #  number of examples to produce.   
    k=1,        
)

# Select the most similar example to the input.
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")


Examples most similar to the input: Who was the father of Mary Ball Washington?


answer: Are follow up questions needed here: Yes.
                    Follow up: Who was the mother of George Washington?
                    Intermediate answer: The mother of George Washington was Mary Ball Washington.
                    Follow up: Who was the father of Mary Ball Washington?
                    Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
                    So the final answer is: Joseph Ball
question: Who was the maternal grandfather of George Washington?
